In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Lib


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from tqdm import tqdm
from torch.utils.data import Dataset
import numpy as np
from torchvision import transforms

import os
import random
import math
import itertools

In [2]:
!pip install -U -qq git+https://github.com/huggingface/diffusers.git
!pip install -qq accelerate transformers ftfy

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.8 MB/s eta 0:00:00


In [3]:
import PIL
import accelerate
from accelerate import Accelerator
from accelerate.logging import get_logger
from accelerate.utils import set_seed

from diffusers import AutoencoderKL, DDPMScheduler, DDIMScheduler, PNDMScheduler, StableDiffusionPipeline, UNet2DConditionModel, DiffusionPipeline
from diffusers.optimization import get_scheduler
from diffusers.pipelines.stable_diffusion import StableDiffusionSafetyChecker
from PIL import Image
import transformers
from transformers import CLIPFeatureExtractor, CLIPTextModel, CLIPTokenizer



In [6]:
import diffusers
print(diffusers.__version__)
pretrained_model_path = "runwayml/stable-diffusion-v1-5"

0.22.0.dev0


In [5]:
text_encoder = CLIPTextModel.from_pretrained(pretrained_model_path, subfolder = "text_encoder")

In [6]:
tokenizer = CLIPTokenizer.from_pretrained(pretrained_model_path, subfolder = "tokenizer")

# Module


In [ ]:
def image_grid(imgs, rows, cols):
  assert len(imgs) == rows * cols

  w,h = imgs[0].size
  grid = Image.new("RGB", size = (cols*w, rows*h))
  grid_w, grid_h = grid.size

  for i, img in enumerate(imgs):
    grid.paste(img, box=(i%cols*w, i//cols*h))

  return grid

Settings for teaching new concept


Get the training images

In [ ]:
urls = [
      "https://huggingface.co/datasets/valhalla/images/resolve/main/2.jpeg",
      "https://huggingface.co/datasets/valhalla/images/resolve/main/3.jpeg",
      "https://huggingface.co/datasets/valhalla/images/resolve/main/5.jpeg",
      "https://huggingface.co/datasets/valhalla/images/resolve/main/6.jpeg",
      ## You can add additional images here
]

In [ ]:

images_path = "/content/drive/MyDrive/Colab Notebooks/Generative_Model/dataset" #@param {type:"string"}
while not os.path.exists(str(images_path)):
  print('The images_path specified does not exist, use the colab file explorer to copy the path :')
  images_path=input("")
save_path = images_path

images = []
for file_path in os.listdir(save_path):
  try:
      image_path = os.path.join(save_path, file_path)
      images.append(Image.open(image_path).resize((512, 512)))
  except:
    print(f"{image_path} is not a valid image, please make sure to remove this file from the directory otherwise the training could fail.")
image_grid(images, 1, len(images))

Output hidden; open in https://colab.research.google.com to view.

# Settings new concept


In [ ]:
wtt= "object"
placeholder_token = "<minhkien>"
init_token = "human"

# Create Dataset

Settings prompt templates

In [ ]:
imagenet_templates_small = [
    "a photo of a {}",
    "a rendering of a {}",
    "a cropped photo of the {}",
    "the photo of a {}",
    "a photo of a clean {}",
    "a photo of a dirty {}",
    "a dark photo of the {}",
    "a photo of my {}",
    "a photo of the cool {}",
    "a close-up photo of a {}",
    "a bright photo of the {}",
    "a cropped photo of a {}",
    "a photo of the {}",
    "a good photo of the {}",
    "a photo of one {}",
    "a close-up photo of the {}",
    "a rendition of the {}",
    "a photo of the clean {}",
    "a rendition of a {}",
    "a photo of a nice {}",
    "a good photo of a {}",
    "a photo of the nice {}",
    "a photo of the small {}",
    "a photo of the weird {}",
    "a photo of the large {}",
    "a photo of a cool {}",
    "a photo of a small {}",
]

imagenet_style_templates_small = [
    "a painting in the style of {}",
    "a rendering in the style of {}",
    "a cropped painting in the style of {}",
    "the painting in the style of {}",
    "a clean painting in the style of {}",
    "a dirty painting in the style of {}",
    "a dark painting in the style of {}",
    "a picture in the style of {}",
    "a cool painting in the style of {}",
    "a close-up painting in the style of {}",
    "a bright painting in the style of {}",
    "a cropped painting in the style of {}",
    "a good painting in the style of {}",
    "a close-up painting in the style of {}",
    "a rendition in the style of {}",
    "a nice painting in the style of {}",
    "a small painting in the style of {}",
    "a weird painting in the style of {}",
    "a large painting in the style of {}",
]

Setup dataset

In [ ]:
#@title Setup the dataset
class TextualInversionDataset(Dataset):
    def __init__(
        self,
        data_root,
        tokenizer,
        learnable_property="object",  # [object, style]
        size=512,
        repeats=100,
        interpolation="bicubic",
        flip_p=0.5,
        set="train",
        placeholder_token="*",
        center_crop=False,
    ):

        self.data_root = data_root
        self.tokenizer = tokenizer
        self.learnable_property = learnable_property
        self.size = size
        self.placeholder_token = placeholder_token
        self.center_crop = center_crop
        self.flip_p = flip_p

        self.image_paths = [os.path.join(self.data_root, file_path) for file_path in os.listdir(self.data_root)]

        self.num_images = len(self.image_paths)
        self._length = self.num_images

        if set == "train":
            self._length = self.num_images * repeats

        self.interpolation = {
            "linear": PIL.Image.LINEAR,
            "bilinear": PIL.Image.BILINEAR,
            "bicubic": PIL.Image.BICUBIC,
            "lanczos": PIL.Image.LANCZOS,
        }[interpolation]

        self.templates = imagenet_style_templates_small if learnable_property == "style" else imagenet_templates_small
        self.flip_transform = transforms.RandomHorizontalFlip(p=self.flip_p)

    def __len__(self):
        return self._length

    def __getitem__(self, i):
        example = {}
        image = Image.open(self.image_paths[i % self.num_images])

        if not image.mode == "RGB":
            image = image.convert("RGB")

        placeholder_string = self.placeholder_token
        text = random.choice(self.templates).format(placeholder_string)

        example["input_ids"] = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.tokenizer.model_max_length,
            return_tensors="pt",
        ).input_ids[0]

        # default to score-sde preprocessing
        img = np.array(image).astype(np.uint8)

        if self.center_crop:
            crop = min(img.shape[0], img.shape[1])
            h, w, = (
                img.shape[0],
                img.shape[1],
            )
            img = img[(h - crop) // 2 : (h + crop) // 2, (w - crop) // 2 : (w + crop) // 2]

        image = Image.fromarray(img)
        image = image.resize((self.size, self.size), resample=self.interpolation)

        image = self.flip_transform(image)
        image = np.array(image).astype(np.uint8)
        image = (image / 127.5 - 1.0).astype(np.float32)

        example["pixel_values"] = torch.from_numpy(image).permute(2, 0, 1)
        return example

# Setup model

In [ ]:
tokenizer = CLIPTokenizer.from_pretrained(pretrained_model_path, subfolder = "tokenizer")

num_add_tokens = tokenizer.add_tokens(placeholder_token)

if num_add_tokens == 0:
  raise ValueError(f"The tokenizer already contains the token {placeholder_token}. Please pass a different"
        " `placeholder_token` that is not already in the tokenizer."
    )

In [ ]:
token_ids = tokenizer.encode(init_token, add_special_tokens=False)

if len(token_ids) > 1:
  raise ValueError("The initializer token must be a single token.")

init_token_id = token_ids[0]
placeholder_id = tokenizer.convert_tokens_to_ids(placeholder_token)

In [ ]:
print(init_token)
print(placeholder_token)

human
<minhkien>


In [ ]:
print(token_ids)
print(init_token_id)
print(placeholder_id)

[2751]
2751
49408


In [ ]:
text_encoder = CLIPTextModel.from_pretrained(pretrained_model_path, subfolder = "text_encoder")

vae = AutoencoderKL.from_pretrained(pretrained_model_path,subfolder = "vae")

unet = UNet2DConditionModel.from_pretrained(pretrained_model_path, subfolder = "unet")

 we have added the **placeholder_token** in the **tokenizer** so we resize the token embeddings here, this will a new embedding vector in the token embeddings for our **placeholder_token**  

In [ ]:
text_encoder.resize_token_embeddings(len(tokenizer))

Embedding(49409, 768)

Initialise the newly added placeholder token with the embeddings of the initializer token

In [ ]:
token_embeds = text_encoder.get_input_embeddings().weight.data
token_embeds[placeholder_id] = token_embeds[init_token_id]

In Textual-Inversion we only train the newly added embedding verctor, so lets freeze ret of the model parameters

In [ ]:
def freeze_parameters(params):
  for param in params:
    param.requires_grad = False

# Freeze vae and unet
freeze_parameters(vae.parameters())
freeze_parameters(unet.parameters())

# freeze all parameters except for the token embeddings in text encoder
params_to_free = itertools.chain(
    text_encoder.text_model.encoder.parameters(),
    text_encoder.text_model.final_layer_norm.parameters(),
    text_encoder.text_model.embeddings.position_embedding.parameters()
)

freeze_parameters(params_to_free)

# Creating training data

Create Dataset and DataLoader

In [ ]:
train_dataset = TextualInversionDataset(
    data_root = save_path,
    tokenizer = tokenizer,
    size = 512,
    placeholder_token = placeholder_token,
    repeats = 100,
    learnable_property = wtt,
    center_crop = False,
    set = "train"
)



In [ ]:
def create_dataloader(train_batch_size = 16):
  return torch.utils.data.DataLoader(train_dataset, batch_size = train_batch_size, shuffle = True)

In [ ]:
dataloader = create_dataloader()

Create noise_scheduler for training

In [ ]:
#scheduler = DDPMScheduler.from_pretrained(pretrained_model_path)
noise_scheduler = DDPMScheduler.from_config(pretrained_model_path, subfolder = "scheduler")



# Training

Define **hyperparameters** for our training if you are not happy with your result, you can tune the **learning_rate** and the **max_train_steps**

## Setting up all training args

In [ ]:
hyperparameters = {
    "learning_rate": 5e-04,
    "scale_lr":True,
    "max_train_steps": 2000,
    "save_steps": 250,
    "train_batch_size": 4,
    "gradient_accumulation_steps": 1,
    "gradient_checkpointing": True,
    "mixed_percision": "fp16",
    "seed": 42,
    "output_dir": f"/content/drive/MyDrive/Colab Notebooks/Generative_Model/concepts/DMK_concept"
}

#!mkdir -p /content/drive/MyDrive/Colab Notebooks/Generative_Model/concepts/cat-toy_concept

## Save your newly created concept to the library of concepts

# New section

In [ ]:
# save_concept_to_public_lib = True
# name_of_concept = "Cat toy"
# hf_token_write = ""

# if(save_concept_to_public_lib):
#   from slugify import slugify
#   from huggingface_hub import HfApi, HfFolder, CommitOperationAdd
#   from huggingface_hub import create_repo

#   repo_id = f"sd-concept-library/{slugify(name_of_concept)}"
#   output_dir = hyperparameters["ouput_dir"]
#   if(not hf_token_write):
#     with open(HfFolder.path_token, 'r') as fin: hf_token = fin.read();
#   else:
#     hf_token = hf_token_write
#   #Join the Concepts Library organization if you aren't part of it already
#   !curl -X POST -H 'Authorization: Bearer '$hf_token -H 'Content-Type: application/json' https://huggingface.co/organizations/sd-concepts-library/share/VcLXJtzwwxnHYCkNMLpSJCdnNFZHQwWywv
#   images_upload = os.listdir("my_concept")
#   image_string = ""
#   repo_id = f"sd-concepts-library/{slugify(name_of_concept)}"
#   for i, image in enumerate(images_upload):
#       image_string = f'''{image_string}![{placeholder_token} {i}](https://huggingface.co/{repo_id}/resolve/main/concept_images/{image})
# '''
#   if(wtt == "style"):
#       what_to_teach_article = f"a `{wtt}`"
#   else:
#       what_to_teach_article = f"an `{wtt}`"
#   readme_text = f'''---

# license: mit
# base_model: {pretrained_model_path}
# ---
# ### {name_of_concept} on Stable Diffusion
# This is the `{placeholder_token}` concept taught to Stable Diffusion via Textual Inversion. You can load this concept into the [Stable Conceptualizer](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/stable_conceptualizer_inference.ipynb) notebook. You can also train your own concepts and load them into the concept libraries using [this notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/sd_textual_inversion_training.ipynb).

# Here is the new concept you will be able to use as {what_to_teach_article}:
# {image_string}
# '''
#   #Save the readme to a file
#   readme_file = open("README.md", "w")
#   readme_file.write(readme_text)
#   readme_file.close()
#   #Save the token identifier to a file
#   text_file = open("token_identifier.txt", "w")
#   text_file.write(placeholder_token)
#   text_file.close()
#   #Save the type of teached thing to a file
#   type_file = open("type_of_concept.txt","w")
#   type_file.write(wtt)
#   type_file.close()
#   operations = [
#     CommitOperationAdd(path_in_repo="learned_embeds.bin", path_or_fileobj=f"{output_dir}/learned_embeds.bin"),
#     CommitOperationAdd(path_in_repo="token_identifier.txt", path_or_fileobj="token_identifier.txt"),
#     CommitOperationAdd(path_in_repo="type_of_concept.txt", path_or_fileobj="type_of_concept.txt"),
#     CommitOperationAdd(path_in_repo="README.md", path_or_fileobj="README.md"),
#   ]
#   create_repo(repo_id,private=True, token=hf_token)
#   api = HfApi()
#   api.create_commit(
#     repo_id=repo_id,
#     operations=operations,
#     commit_message=f"Upload the concept {name_of_concept} embeds and token",
#     token=hf_token
#   )
#   api.upload_folder(
#     folder_path=save_path,
#     path_in_repo="concept_images",
#     repo_id=repo_id,
#     token=hf_token
#   )

## Set up the pipeline

In [ ]:
# from diffusers import DPMSolverMultistepScheduler
# pipe = StableDiffusionPipeline.from_pretrained(
#     hyperparameters["output_dir"],
#     scheduler=DPMSolverMultistepScheduler.from_pretrained(hyperparameters["output_dir"], subfolder="scheduler"),
#     torch_dtype=torch.float16,
# ).to("cuda")

## New section

In [ ]:
# #@title Run the Stable Diffusion pipeline
# #@markdown Don't forget to use the placeholder token in your prompt

# prompt = "a \u003Ccat-toy> inside ramen-bowl" #@param {type:"string"}

# num_samples = 2 #@param {type:"number"}
# num_rows = 1 #@param {type:"number"}

# all_images = []
# for _ in range(num_rows):
#     images = pipe([prompt] * num_samples, num_inference_steps=30, guidance_scale=7.5).images
#     all_images.extend(images)

# grid = image_grid(all_images, num_rows, num_samples)
# grid

In [ ]:
save_path

'/content/drive/MyDrive/Colab Notebooks/Generative_Model/dataset'

# Training function

In [ ]:
logger = get_logger(__name__)

def save_progress(text_encoder, placeholder_token_id, accelerator, save_path):
  logger.info("Saving embeddings")
  learned_embeds = accelerator.unwrap_model(text_encoder).get_input_embeddings().weight[placeholder_token_id]
  learned_embeds_dict = {placeholder_token: learned_embeds.detach().cpu()}
  torch.save(learned_embeds_dict, save_path)


def training_function(text_encoder, unet, vae):

  train_batch_size = hyperparameters["train_batch_size"]
  gradient_accumulation_steps = hyperparameters["gradient_accumulation_steps"]
  learning_rate = hyperparameters["learning_rate"]
  max_train_steps = hyperparameters["max_train_steps"]
  output_dir = hyperparameters["output_dir"]
  gradient_checkpointing = hyperparameters["gradient_checkpointing"]

  accelerator = Accelerator(
      gradient_accumulation_steps = gradient_accumulation_steps,
      #mixed_precision = hyperparameters["mixed_precision"]
  )

  train_dataloader = create_dataloader(train_batch_size = train_batch_size)

  if hyperparameters["scale_lr"]:
    learning_rate = (
        learning_rate * gradient_accumulation_steps * train_batch_size * accelerator.num_processes
    )

  optimizer = torch.optim.AdamW(
      text_encoder.get_input_embeddings().parameters(), lr = learning_rate
  )

  text_encoder, optimizer, train_dataloader = accelerator.prepare(text_encoder, optimizer, train_dataloader)

  weight_dtype = torch.float32
  if accelerator.mixed_precision == "fp16":
      weight_dtype = torch.float16
  elif accelerator.mixed_precision == "bf16":
      weight_dtype = torch.bfloat16


  # Move vae, unet to device
  vae.to(accelerator.device,dtype = weight_dtype )
  unet.to(accelerator.device, dtype = weight_dtype)

  #Keep vae, unet in eval model as we dont train these
  vae.eval()
  unet.eval()

  num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)
  num_train_epochs = math.ceil(max_train_steps / num_update_steps_per_epoch)

  #Train

  total_batch_size = train_batch_size * accelerator.num_processes * gradient_accumulation_steps
  print(total_batch_size)
  logger.info("***** Running training *****")
  logger.info(f"  Num examples = {len(train_dataset)}")
  logger.info(f"  Instantaneous batch size per device = {train_batch_size}")
  logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
  logger.info(f"  Gradient Accumulation steps = {gradient_accumulation_steps}")
  logger.info(f"  Total optimization steps = {max_train_steps}")

  progress_bar = tqdm(range(max_train_steps), disable = not accelerator.is_local_main_process)
  progress_bar.set_description("Step")
  global_step = 0

  for epoch in range(num_train_epochs):
    text_encoder.train()
    for step, batch in enumerate(train_dataloader):
      with accelerator.accumulate(text_encoder):
        latents = vae.encode(batch["pixel_values"].to(dtype = weight_dtype)).latent_dist.sample().detach()
        latents = latents * 0.18215

        #Sample noise that we will add to the latents
        noise = torch.randn_like(latents)
        bsz = latents.shape[0]

        #Sample a random timestep for each image
        timesteps = torch.randint(0, noise_scheduler.num_train_timesteps, (bsz,), device = latents.device).long()

        # Add noise to the latents according to the noise magnitude at each timestep
        # This is the forward diffusion process
        noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

        # Get text embedding for conditioning
        encoder_hidden_state = text_encoder(batch["input_ids"])[0]

        # Predict noise image residual
        noise_pred = unet(noisy_latents, timesteps, encoder_hidden_state.to(dtype = weight_dtype)).sample

         # Get the target for loss depending on the prediction type
        if noise_scheduler.config.prediction_type == "epsilon":
          target = noise
        elif noise_scheduler.config.prediction_type == "v_prediction":
          target = noise_scheduler.get_velocity(latents, noise, timesteps)
        else:
          raise ValueError(f"Unknown prediction type {noise_scheduler.config.prediction_type}")


        loss = F.mse_loss(noise_pred, target, reduction = "none").mean([1,2,3]).mean()

        accelerator.backward(loss)

        if accelerator.num_processes > 1:
          grads = text_encoder.module_get_input_embeddings().weight.grad

        else:
          grads = text_encoder.get_input_embeddings().weight.grad

        # Get the index for tokens that we want to zero the grads for
        index_grads_to_zero = torch.arange(len(tokenizer)) != placeholder_id
        grads.data[index_grads_to_zero, :] = grads.data[index_grads_to_zero, :].fill_(0)

        optimizer.step()
        optimizer.zero_grad()

        # Checks if the accelarator has performed an optimization step behind scenes
      if accelerator.sync_gradients:
        progress_bar.update(1)
        global_step += 1
        if global_step % hyperparameters["save_steps"] == 0:
          save_path = os.path.join(output_dir, f"learned_embeds-step-{global_step}.bin")
          save_progress(text_encoder, placeholder_id, accelerator, save_path)


      logs = {"loss": loss.detach().item()}
      progress_bar.set_postfix(**logs)

      if global_step >= max_train_steps:
        break

    accelerator.wait_for_everyone()


  if accelerator.is_main_process:
    pipeline = StableDiffusionPipeline.from_pretrained(
        pretrained_model_path,
        text_encoder = accelerator.unwrap_model(text_encoder),
        vae = vae,
        unet = unet,
        tokenizer = tokenizer,
        #scheduler = DPMSolverMultistepScheduler.from_pretrained(pretrained_model_paths, subfoler = "scheduler"),
        #safety_checker = StableDiffusionSafetyChecker.from_pretrained("CompVis/stable-diffusion-safety-checker"),
        #feature_extractor = CLIPFeatureExtractor.from_pretrained("openai/clip-vit-base-patch32")
    )

    pipeline.save_pretrained(output_dir)

   #learned_embeds = accelerator.unwrap_model(text_encoder).get_input_embeddings().weight[placeholder_id]
    #learned_embeds_dict = {placeholder_token: learned_embeds.detach().cpu()}

    #torch.save(learned_embeds_dict, os.path.join(output_dir, "learned_embeds.bin"))

    save_path = os.path.join(output_dir, f"learned_embeds.bin")
    save_progress(text_encoder, placeholder_id, accelerator, save_path)



In [ ]:
  import accelerate
  accelerate.notebook_launcher(training_function, args = (text_encoder, unet, vae))

  for param in itertools.chain(unet.parameters(), text_encoder.parameters()):
    if param.grad is not None:
      del param.grad
    torch.cuda.empty_cache()

Launching training on one GPU.
4



Step:   0%|          | 0/2000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/diffusers/configuration_utils.py:134: FutureWarning: Accessing config attribute `num_train_timesteps` directly via 'DDPMScheduler' object attribute is deprecated. Please access 'num_train_timesteps' over 'DDPMScheduler's config object instead, e.g. 'scheduler.config.num_train_timesteps'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)

Step: 100%|██████████| 2000/2000 [27:16<00:00,  1.29it/s, loss=0.231] 

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
Step: 100%|██████████| 2000/2000 [27:40<00:00,  1.20it/s, loss=0.231]

# Inference

In [ ]:
from diffusers.schedulers import DPMSolverMultistepScheduler
pipe = StableDiffusionPipeline.from_pretrained(
    hyperparameters["output_dir"],
    scheduler = DPMSolverMultistepScheduler.from_pretrained(hyperparameters["output_dir"], subfolder = "scheduler"),
    torch_dtype = torch.float16).to("cuda")

In [ ]:
from torchvision.utils import ImageColor

path = "/content/drive/MyDrive/Colab Notebooks/Generative_Model/dataset/DMK"


In [ ]:
def save_images(images, path, concept_name):
  for i,_ in enumerate(images):
    img = images[i]
    img = img.save(path + "/" + concept_name + "_" + str(i) + ".jpeg")

In [ ]:
#@title Run the Stable Diffusion pipeline
#@markdown Don't forget to use the placeholder token in your prompt

prompt = "<minhkien> go to the France"
num_samples = 3  # @param {type:"number"}
num_rows = 4

# prevent safety checking
# def dummy(images, **kwargs):
#     return images, False
# pipe.safety_checker = dummy

all_images = []
for _ in range(num_rows):
    images = pipe([prompt] * num_samples, num_inference_steps=50, guidance_scale=7.5).images
    all_images.extend(images)



  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
save_images(all_images, path, "generate_France")